# FACT_CONSUMPTION_MONTHLY - Monthly Aggregated Consumption Table

Monthly aggregated consumption and revenue metrics for month-over-month analysis in the Genie Space.

**Note:** This table is populated by aggregating from `fact_consumption_daily`.

## 1. Create Table Schema

In [ ]:
# %sql
# -- Drop table if exists (uncomment if needed)
# -- DROP TABLE IF EXISTS ankit_yadav.fact_consumption_monthly;

CREATE TABLE IF NOT EXISTS ankit_yadav.fact_consumption_monthly (
  year_month STRING COMMENT 'Year and month (e.g., 2025-01)',
  month_start_date DATE COMMENT 'First day of the month',
  account_id STRING NOT NULL COMMENT 'Foreign key to dim_accounts',
  workspace_id STRING COMMENT 'Databricks workspace identifier',
  platform STRING COMMENT 'Cloud platform (AWS, Azure, GCP)',
  sku STRING COMMENT 'Product SKU',
  account_name STRING COMMENT 'Customer account name',
  account_executive STRING COMMENT 'Account Executive name',
  account_executive_manager STRING COMMENT 'AE Manager name',
  workspace_name STRING COMMENT 'Workspace display name',
  total_revenue_dollars FLOAT COMMENT 'Total revenue for the month in dollars',
  total_dbus_consumed FLOAT COMMENT 'Total DBUs consumed for the month',
  avg_list_price_per_dbu FLOAT COMMENT 'Average list price per DBU',
  consumption_days_count INT COMMENT 'Number of days with consumption in the month',
  last_updated_timestamp TIMESTAMP COMMENT 'Last time this record was updated'
)
COMMENT 'Monthly aggregated consumption and revenue metrics for month-over-month analysis';

## 2. Populate from Daily Data

Aggregate daily consumption data into monthly summaries.

In [ ]:
# %sql
-- Clear existing data for demo accounts
DELETE FROM ankit_yadav.fact_consumption_monthly WHERE account_id LIKE 'DEMO%';

-- Aggregate from daily to monthly
INSERT INTO ankit_yadav.fact_consumption_monthly
SELECT
  DATE_FORMAT(DATE_TRUNC('MONTH', consumption_date), 'yyyy-MM') as year_month,
  DATE_TRUNC('MONTH', consumption_date) as month_start_date,
  account_id,
  workspace_id,
  platform,
  sku,
  MAX(account_name) as account_name,
  MAX(account_executive) as account_executive,
  MAX(account_executive_manager) as account_executive_manager,
  MAX(workspace_name) as workspace_name,
  ROUND(SUM(revenue_dollars), 2) as total_revenue_dollars,
  ROUND(SUM(dbus_consumed), 2) as total_dbus_consumed,
  ROUND(AVG(list_price_per_dbu), 4) as avg_list_price_per_dbu,
  COUNT(DISTINCT consumption_date) as consumption_days_count,
  CURRENT_TIMESTAMP() as last_updated_timestamp
FROM ankit_yadav.fact_consumption_daily
WHERE account_id LIKE 'DEMO%'
GROUP BY
  DATE_TRUNC('MONTH', consumption_date),
  account_id,
  workspace_id,
  platform,
  sku;

## 3. Verify Data

In [ ]:
# %sql
-- Sample monthly data
SELECT * FROM ankit_yadav.fact_consumption_monthly 
WHERE account_id = 'DEMO001'
ORDER BY month_start_date DESC, sku;

In [ ]:
# %sql
-- Monthly revenue by account
SELECT 
  year_month,
  account_name,
  account_executive_manager,
  ROUND(SUM(total_revenue_dollars), 2) as monthly_revenue,
  ROUND(SUM(total_dbus_consumed), 2) as monthly_dbus,
  SUM(consumption_days_count) as consumption_days
FROM ankit_yadav.fact_consumption_monthly
GROUP BY year_month, account_name, account_executive_manager
ORDER BY year_month DESC, monthly_revenue DESC;

In [ ]:
# %sql
-- Monthly totals by AEM
SELECT 
  year_month,
  account_executive_manager,
  COUNT(DISTINCT account_id) as account_count,
  ROUND(SUM(total_revenue_dollars), 2) as total_revenue,
  ROUND(SUM(total_dbus_consumed), 2) as total_dbus
FROM ankit_yadav.fact_consumption_monthly
GROUP BY year_month, account_executive_manager
ORDER BY year_month DESC, total_revenue DESC;

In [ ]:
# %sql
-- Monthly revenue by SKU
SELECT 
  year_month,
  sku,
  ROUND(SUM(total_revenue_dollars), 2) as total_revenue,
  ROUND(SUM(total_dbus_consumed), 2) as total_dbus,
  COUNT(DISTINCT account_id) as account_count
FROM ankit_yadav.fact_consumption_monthly
GROUP BY year_month, sku
ORDER BY year_month DESC, total_revenue DESC;

In [ ]:
# %sql
-- Record count summary
SELECT 
  COUNT(*) as total_records,
  COUNT(DISTINCT account_id) as unique_accounts,
  COUNT(DISTINCT year_month) as unique_months,
  MIN(month_start_date) as earliest_month,
  MAX(month_start_date) as latest_month,
  ROUND(SUM(total_revenue_dollars), 2) as total_revenue
FROM ankit_yadav.fact_consumption_monthly;

In [ ]:
# %sql
-- Month-over-month growth analysis
WITH monthly_totals AS (
  SELECT 
    year_month,
    month_start_date,
    SUM(total_revenue_dollars) as monthly_revenue
  FROM ankit_yadav.fact_consumption_monthly
  GROUP BY year_month, month_start_date
)
SELECT 
  year_month,
  ROUND(monthly_revenue, 2) as monthly_revenue,
  ROUND(LAG(monthly_revenue) OVER (ORDER BY month_start_date), 2) as prev_month_revenue,
  ROUND(
    (monthly_revenue - LAG(monthly_revenue) OVER (ORDER BY month_start_date)) / 
    LAG(monthly_revenue) OVER (ORDER BY month_start_date) * 100, 2
  ) as mom_growth_pct
FROM monthly_totals
ORDER BY month_start_date DESC;

In [ ]:
# %sql
-- Top 5 accounts by monthly revenue (latest month)
SELECT 
  account_name,
  account_executive,
  account_executive_manager,
  ROUND(SUM(total_revenue_dollars), 2) as monthly_revenue,
  ROUND(SUM(total_dbus_consumed), 2) as monthly_dbus
FROM ankit_yadav.fact_consumption_monthly
WHERE year_month = (SELECT MAX(year_month) FROM ankit_yadav.fact_consumption_monthly)
GROUP BY account_name, account_executive, account_executive_manager
ORDER BY monthly_revenue DESC
LIMIT 5;